# Artifact Rejection - Brief Testing

In [3]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import scipy
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import yaml
import re

import hypnogram as hp
from ecephys.utils import replace_outliers
import ecephys.plot as eplt
import ecephys.signal.timefrequency as tfr
import ecephys.signal.utils as kd
import ecephys.signal.kd_plotting as kp
import ecephys.signal.kd_pax as kpx
import tdt_xarray as tx

import adtk
from adtk.detector import GeneralizedESDTestAD
from adtk.visualization import plot

bands_def = dict(delta=(0.5, 4.0), theta=(4.1, 8), sigma=(11,16), beta = (13, 20), low_gamma = (40, 55), high_gamma = (65, 80))

kd_ref = {}
kd_ref['echans'] = [1,2,3]
kd_ref['fchans']=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
kd_ref['analysis_root'] = Path('/Volumes/opto_loc/Data/ACHR_PROJECT_MATERIALS/ACHR_2/ACHR_2-analysis-data')
kd_ref['tank_root'] = Path('/Volumes/opto_loc/Data/ACHR_2_TANK')

# Getting the data we need

In [71]:
#get a single channel of data as a dataframe with datetime column
def bp_df(xr_spg, chan, band_range, t1=None, t2=None):
    if t1 is not None: 
        xr_spg = xr_spg.sel(time=slice(t1, t2))
    
    try: 
        xr_spg = xr_spg.swap_dims({'time': 'datetime'})
    except:
        pass
    xr_spg = xr_spg.sel(channel=chan)
    xr_bp = kd.get_bandpower(xr_spg, band_range)
    df_bp = xr_bp.to_dataframe(name='band_power')
    df_bp = df_bp.drop(labels=['channel','timedelta','time'], axis=1)
    return df_bp

In [4]:
bl2p = '/Volumes/opto_loc/Data/ACHR_2/ACHR_2_TANK/ACHR_2-exp2-bl' 

In [5]:
art = {}
art['ebl'], art['ebl_spg'] = kd.get_data_spg(bl2p, store='EEG_', t1=0, t2=7200, channel=kd_ref['echans'])
art['fbl'], art['fbl_spg'] = kd.get_data_spg(bl2p, store='LFP_', t1=0, t2=7200, channel=kd_ref['fchans'])

# Artifact Hypnograms
lfp_hyp = hp.load_visbrain_hypnogram("/Volumes/opto_loc/Data/ACHR_PROJECT_MATERIALS/artifact_debug/ex2-bl1-LFP.txt")
eeg_hyp = hp.load_visbrain_hypnogram("/Volumes/opto_loc/Data/ACHR_PROJECT_MATERIALS/artifact_debug/ex2-bl1-EEG.txt")

read from t=0s to t=7929.87s
Using 1017.2526 Hz as SEV sampling rate for EEG_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset
read from t=0s to t=7929.87s
Using 1017.2526 Hz as SEV sampling rate for LFP_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset


In [42]:
lfbp = kd.get_bp_set(art['fbl_spg'], bands_def)

In [43]:
lfbp = lfbp.sel(channel=2)

In [60]:
d = lfbp['delta']

In [62]:
d = d.to_dataframe()

In [63]:
d

,channel,timedelta,datetime,delta
time,,,,
1.999995,2,0 days 00:00:01.999994839,2021-10-21 10:01:19.999993839,17886.109375
5.000233,2,0 days 00:00:05.000232858,2021-10-21 10:01:23.000231858,11623.974609
8.000471,2,0 days 00:00:08.000470876,2021-10-21 10:01:26.000469876,17113.660156
11.000709,2,0 days 00:00:11.000708895,2021-10-21 10:01:29.000707895,27628.343750
14.000947,2,0 days 00:00:14.000946913,2021-10-21 10:01:32.000945913,17461.017578
...,...,...,...,...
7184.569811,2,0 days 01:59:44.569811260,2021-10-21 12:01:02.569810260,98246.226562
7187.570049,2,0 days 01:59:47.570049279,2021-10-21 12:01:05.570048279,68581.757812
7190.570287,2,0 days 01:59:50.570287297,2021-10-21 12:01:08.570286297,101450.335938


In [84]:
delta = (0.5, 4)
bp2 = bp_df(art['fbl_spg'], chan=2, band_range=delta, t1=0, t2=4000)

In [85]:
bp2

,band_power
datetime,
2021-10-21 10:01:19.999993839,16086.372070
2021-10-21 10:01:23.000231858,11571.850586
2021-10-21 10:01:26.000469876,17051.394531
2021-10-21 10:01:29.000707895,27467.548828
2021-10-21 10:01:32.000945913,17428.160156
...,...
2021-10-21 11:07:44.316082480,6773.974121
2021-10-21 11:07:47.316320499,11325.466797
2021-10-21 11:07:50.316558517,15417.276367


In [29]:
lf2 = x2df(art['fbl'], chan=2)

In [33]:
lf2_cut = x2df(art['fbl'], chan=2, t1=3000, t2=4000)

# ADTK implementations

In [35]:
esd_ad = GeneralizedESDTestAD(alpha=0.3)
anomalies = esd_ad.fit_detect(lf2_cut)

In [38]:
esd_ad = GeneralizedESDTestAD(alpha=0.01)
anomalies2 = esd_ad.fit_detect(lf2_cut)

In [95]:
esd_ad = GeneralizedESDTestAD(alpha=0.0000001)
anomaliesbp = esd_ad.fit_detect(bp2)

In [59]:
anomalies2_2 = anomalies2 == False

In [37]:
from adtk.visualization import plot
plot(lf2_cut, anomaly=anomalies, ts_linewidth=1, ts_markersize=3, anomaly_markersize=5, anomaly_color='red', anomaly_tag="marker");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
from adtk.visualization import plot
plot(lf2_cut, anomaly=anomalies2, ts_linewidth=1, ts_markersize=3, anomaly_markersize=5, anomaly_color='red', anomaly_tag="marker");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
from adtk.visualization import plot
plot(bp2, anomaly=anomaliesbp, ts_linewidth=1, ts_markersize=3, anomaly_markersize=5, anomaly_color='red', anomaly_tag="marker");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exploring the distribution of the RAW data

First we need to be able to actually explore how the values are distrubuted to find out where the artifacts may be

In [97]:
plt.close('all')
band='delta'
spg = art['fbl_spg']
chans = [2,4]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(spg, bands_def, band, hyp=lfp_hyp, channel=chan, start_time=0, end_time=7200, title=band.capitalize()+" Power, LFP-"+str(chan)+", ACHR_2 EXPERIMENT-2-bl")
    b.set_ylim(0, 400000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
df = a2x1['f'].to_dataframe()

In [ ]:
df1 = a2x1['f'].sel(channel=1).swap_dims({'time': 'datetime'}).to_dataframe()

In [ ]:
df1.min()

LFP_   -9559.318359
dtype: float32

In [ ]:
df1 = df1.drop(labels=['channel','timedelta','time'], axis=1)

In [ ]:
.drop(labels=['channel','timedelta','datetime'], axis=1)

In [ ]:
ax = df1.plot.hist(bins=1000)
ax.set_title('LFP-1 Raw data distribution, exp-1 full recording')
ax.set_xlabel('value of sample in mV')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'value of sample in mV')

# Exploring distribution of spectral calculations

In [ ]:
a2x1['fbp'] = kd.get_bp_set(a2x1['fspg'].sel(channel=1), bands_def)

In [ ]:
delta1 = a2x1['fbp'].delta.to_dataframe().drop(labels=['channel','timedelta','datetime'], axis=1)

In [ ]:
dx = delta1.to_xarray().delta

In [ ]:
m1 = delta1<200000
d1m = delta1[m1]

In [ ]:
delta1.plot.hist(bins=2000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:ylabel='Frequency'>

# Actually rejecting data from our xsets:

In [ ]:
ch1 = a2x1['f'].sel(channel=1).values
sd1 = np.std(ch1)


upper = (5*sd1)
lower = -(5*sd1)


# FALSE where there are artifacts
m1 = np.logical_and(ch1<upper, ch1>lower)
 
# TRUE where there are artifacts
m2 = np.logical_or(ch1>upper, ch1<lower) 

In [ ]:
a2x1['f'].sel(channel=1).values[m2] = None
a2x1['fspg'] = kd.get_spextrogram(a2x1['f'])